In [1]:
import os
import glob
import datetime as dt
import numpy as np
import xarray as xr
import pandas as pd
from sublimpy import utils, extrautils
import matplotlib.pyplot as plt
import altair as alt
alt.data_transformers.enable('json')
import act
import swifter

Open tidy dataset

In [2]:
sos_tidy_fn = f"../paper1/process_slow_data/tidy_df_20221101_20230619_planar_fit_multiplane_q7_flags360_pf10.parquet"
tidy_df = pd.read_parquet(sos_tidy_fn)
# Convert data timezone to local and clean up data on the ends
# convert time column to datetime
tidy_df['time'] = pd.to_datetime(tidy_df['time'])
tidy_df = utils.modify_df_timezone(tidy_df, 'UTC', 'US/Mountain')
# tidy_df = tidy_df[tidy_df.time > '20230301']
tidy_df = tidy_df[tidy_df.time > '20221130']
tidy_df = tidy_df[tidy_df.time < '20230509']
tidy_df.time.min(), tidy_df.time.max()

(Timestamp('2022-11-30 00:30:00'), Timestamp('2023-05-08 23:30:00'))

classify synoptic winds

In [3]:
USERNAME = os.getenv("ARM_USERNAME")
TOKEN = os.getenv("ARM_TOKEN")
SAIL_DATA_STREAM = 'gucdlprofwind4newsM1.c1'
DATA_STREAM_FILEEXT = '.nc'
startdate = "2022-11-30"
enddate = "2023-06-20"
output_dir = os.path.join("/Users/elischwat/Development/data/sublimationofsnow/", SAIL_DATA_STREAM)


In [4]:
# act.discovery.download_arm_data(USERNAME, TOKEN, SAIL_DATA_STREAM, startdate, enddate, output=output_dir)

In [5]:
dl_w_prof_files = glob.glob("/Users/elischwat/Development/data/sublimationofsnow/gucdlprofwind4newsM1.c1/*.nc")

In [6]:
prof_ds = act.io.read_arm_netcdf(dl_w_prof_files)
prof_ds.time.min(), prof_ds.time.max()

(<xarray.DataArray 'time' ()> Size: 8B
 array('2022-11-30T00:00:39.646250000', dtype='datetime64[ns]'),
 <xarray.DataArray 'time' ()> Size: 8B
 array('2023-06-15T23:59:20.789108000', dtype='datetime64[ns]'))

In [7]:
prof_df = prof_ds.to_dataframe()
prof_df = utils.modify_df_timezone(prof_df.reset_index(), 'UTC', 'US/Mountain').set_index('time')

In [8]:
hourly_mean_prof_df = prof_df.query("height > 950").query("height < 1100")[['wind_speed', 'wind_direction']].reset_index().set_index('time').groupby(pd.Grouper(freq='60min')).mean()

In [9]:
synoptic_winds_30min = hourly_mean_prof_df.resample('30min').mean().ffill()
along_valley_wind_times = synoptic_winds_30min[
    (synoptic_winds_30min.wind_direction > 299)
    & (synoptic_winds_30min.wind_direction < 345 )
].index.values
cross_valley_wind_times = synoptic_winds_30min[
    (synoptic_winds_30min.wind_direction > 202)
    & (synoptic_winds_30min.wind_direction < 248 )
].index.values

In [10]:
along_valley_wind_times = pd.DataFrame(along_valley_wind_times).set_index(0)
along_valley_wind_times.index.name = 'time'
along_valley_wind_times = along_valley_wind_times.assign(category = 'along-valley')
cross_valley_wind_times = pd.DataFrame(cross_valley_wind_times).set_index(0)
cross_valley_wind_times.index.name = 'time'
cross_valley_wind_times = cross_valley_wind_times.assign(category = 'cross-valley')

cross_valley_wind_times

,category
time,
2022-12-01 17:00:00,cross-valley
2022-12-01 17:30:00,cross-valley
2022-12-01 18:00:00,cross-valley
2022-12-01 18:30:00,cross-valley
2022-12-01 19:00:00,cross-valley
...,...
2023-06-15 10:30:00,cross-valley
2023-06-15 13:00:00,cross-valley
2023-06-15 13:30:00,cross-valley


In [11]:
synoptic_winds_pdf_data = pd.concat([
        synoptic_winds_30min[synoptic_winds_30min.index.isin(cross_valley_wind_times.index)].assign(synoptic_wind = 'cross valley'),
        synoptic_winds_30min[synoptic_winds_30min.index.isin(along_valley_wind_times.index)].assign(synoptic_wind = 'along valley')
    ])

In [12]:
fast_synoptic_wind_times = synoptic_winds_pdf_data.query("wind_speed <= 10").assign(synoptic_wind_speed = '<= 10 m/s')[['synoptic_wind_speed']]
slow_synoptic_wind_times = synoptic_winds_pdf_data.query("wind_speed > 10").assign(synoptic_wind_speed = '> 10 m/s')[['synoptic_wind_speed']]

# Scaling, following Stull

In [13]:
scale_df = pd.DataFrame()
for H in [2,3,5,10,15,20]:
    src = tidy_df[tidy_df.variable.isin([
        f"w_w__{H}m_c",
        f"u*_{H}m_c",
    ])].pivot_table(index='time', columns='variable', values='value')
    src[f'scale'] = (
        src[f'w_w__{H}m_c'] / src[f'u*_{H}m_c']**2
    )
    src[f'height'] = H
    scale_df = pd.concat([scale_df, src[['height', 'scale']]])

In [14]:
scale_df = pd.concat([
    scale_df.join(along_valley_wind_times, how='inner'),
    scale_df.join(cross_valley_wind_times, how='inner')
])
scale_df = pd.concat([
    scale_df.join(fast_synoptic_wind_times, how='inner'),
    scale_df.join(slow_synoptic_wind_times, how='inner')
])

In [15]:
alt.layer(
        alt.Chart().mark_circle(size=150, color='#1f77b4').encode(
            alt.Color('height:O'),
            alt.X('category').axis(labelAngle=30).title(None),
            alt.XOffset('height:O'),
            alt.Y('median(scale):Q').title("w'w' / u*"),
        ),
        alt.Chart().mark_line(color='#1f77b4', strokeWidth=2).encode(
            alt.Color('height:O'),
            alt.X('category'),
            alt.XOffset('height:O'),
            alt.Y('q1(scale)'),
            alt.Y2('q3(scale):Q'),
        ),
        alt.Chart().transform_calculate(
            y = '2.5'
        ).mark_rule(color='black').encode(alt.Y('y:Q')),
        alt.Chart().transform_calculate(
            y = '1.7'
        ).mark_rule(color='black').encode(alt.Y('y:Q')),
        data = scale_df
).transform_filter(alt.datum.height == 5).facet(column=alt.Column('synoptic_wind_speed:N', title = 'Synoptic Wind Speed'))

alt.FacetChart(...)

In [16]:
alt.layer(
        alt.Chart().mark_circle(size=150, color='#1f77b4').encode(
            alt.Color('height:O'),
            alt.X('category').axis(labelAngle=30).title(None),
            alt.XOffset('height:O'),
            alt.Y('median(scale):Q').title("w'w' / u*"),
        ),
        alt.Chart().mark_line(color='#1f77b4', strokeWidth=2).encode(
            alt.Color('height:O'),
            alt.X('category'),
            alt.XOffset('height:O'),
            alt.Y('q1(scale)'),
            alt.Y2('q3(scale):Q'),
        ),
        alt.Chart().transform_calculate(
            y = '2.5'
        ).mark_rule(color='black').encode(alt.Y('y:Q')),
        alt.Chart().transform_calculate(
            y = '1.7'
        ).mark_rule(color='black').encode(alt.Y('y:Q')),
        data = scale_df
).properties().facet(column=alt.Column('synoptic_wind_speed:N', title = 'Synoptic Wind Speed'))

alt.FacetChart(...)

In [24]:
alt.layer(
        alt.Chart().mark_point(size=150, color='#1f77b4').encode(
            alt.X('category:N'),
            alt.Shape('category:N'),
            alt.Color('category:N'),
            alt.Y('median(scale):Q').title("w'w' / u*"),
        ),
        alt.Chart().mark_line(color='#1f77b4', strokeWidth=2).encode(
            alt.X('category:N').axis(values=[]).title(''),
            alt.Color('category:N'),
            alt.Y('q1(scale)'),
            alt.Y2('q3(scale):Q'),
            detail = 'category'
        ),
        alt.Chart().transform_calculate(
            y = '2.5'
        ).mark_rule(color='black').encode(alt.Y('y:Q')),
        alt.Chart().transform_calculate(
            y = '1.7'
        ).mark_rule(color='black').encode(alt.Y('y:Q')),
        data = scale_df.query("synoptic_wind_speed == '> 10 m/s'")
).properties(height=200).facet(column=alt.Column('height:O', title = 'Height AGL (m)'))


alt.FacetChart(...)